In [16]:
from price_impact.modelling.Futures import FutureContract, Spot
from price_impact.modelling.Futures import future_chain, future_active_months, expiry_rules, roll_lookback_weeks, ticker_descriptions
from price_impact.modelling.backtest_analysis import BacktestResults, BacktestSettings
from price_impact import tickers

import json
import pandas as pd 
import calendar
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
import os

## Info about futures

In [17]:
def format_lines(input_string, line_length_limit):
    words = input_string.split()
    lines = []
    current_line = []

    for word in words:
        if len(' '.join(current_line + [word])) <= line_length_limit:
            current_line.append(word)
        else:
            lines.append(' '.join(current_line))
            current_line = [word]
    
    if current_line:
        lines.append(' '.join(current_line))

    if len(lines)<2:
        return lines[0]
    
    out_line = r"\makecell{"
    out_line += r'\\'.join(lines)
    out_line += "}"
    
    return out_line


def format_col_name(col):
    out_col = "\makecell{"
    out_col += r'\\'.join(col.split('_'))
    out_col += "}"
    return out_col

In [18]:
fut_chain = future_chain.copy()
fut_chain['bbg_ticker'] = fut_chain['tick_code'].apply(lambda x: Spot(x).bbg_ticker)
fut_chain = fut_chain[['tick_code', 'bbg_ticker', 'future_months', 'roll_lookback_weeks', 'expiry_rule', 'description', 'asset_class']]
fut_chain.loc[:, 'future_months'] = fut_chain['future_months'].apply(lambda x: ''.join(json.loads(x.replace("'", '"'))))

fut_chain.loc[:, 'expiry_rule'] = fut_chain['expiry_rule'].fillna('third_friday').str.replace('_', ' ')

# create rules lookup table
rules = fut_chain.iloc[:, 4].unique()
rules = {idx: rule for idx, rule in enumerate(rules)}
rules_rev = {rule: idx for idx, rule in rules.items()}

fut_chain['expiry_rule'] = fut_chain['expiry_rule'].map(rules_rev)

fut_chain['future_months'] = fut_chain['future_months'].apply(lambda x: x if len(x) < 6 else rf"\makecell{{{x[:6]}\\{x[6:]}}}")

fut_chain['description'] = fut_chain['description'].apply(lambda x: x.replace('&', r'\&').replace('#', '\#'))
fut_chain['description'] = fut_chain['description'].apply(lambda x: format_lines(x, 20))

fut_chain['asset_class'] = fut_chain['asset_class'].apply(lambda x: x.replace('&', r'\&').replace('#', '\#'))

fut_chain = fut_chain.sort_values(by=['asset_class', 'tick_code']).reset_index(drop=True)


fut_chain.columns = list(map(format_col_name, fut_chain.columns))

file_path = './tables/appendix/futures.tex'
caption = "Futures used"
label = "table:apx_futures"

with pd.option_context("max_colwidth", 1000):
    fut_chain.to_latex(file_path, index=False, float_format="%.1f", longtable=True, caption=caption, label=label, escape=False)

rules_df = pd.Series(rules_rev).to_frame().reset_index()
rules_df.columns = ['Expiration Rule', 'Rule Key']
rules_df.to_latex('./tables/appendix/expiration_rules.tex', index=False, escape=False)

months = 'FGHJKMNQUVXZ'
months = {m: n for m, n in zip(months, calendar.month_name[1:])}
months = pd.Series(months).to_frame().reset_index()
months.columns = ['Month Key', 'Month']
months.to_latex('./tables/appendix/months.tex', index=False, escape=False)

### Full results set

In [19]:
train_months, test_months = 6, 3

settings = BacktestSettings(
    train_len=train_months,
    test_len=test_months,
    bin_freq='10s',
    forecast_horizon_s=10,
    smoothing_str='sma20',
    decay=3600
)

results_list = []


def get_stat_dict(spot):

    backtest_results = BacktestResults(spot, settings)
    stat_dict = backtest_results.statistics_dict
    stat_dict['spot'] = spot
    return stat_dict


with ThreadPoolExecutor(10) as executor:
    results_list = executor.map(get_stat_dict, tickers)

results_list = [res for res in results_list if res is not None]

In [20]:

df = pd.DataFrame(results_list).sort_values("pct_curr_best_is_next_best")
df['spot'] = df['spot'].apply(lambda x: Spot(x))
df['asset_class'] = df['spot'].apply(lambda x: x.asset_class)

df = df[df['spot'] != 'EI']
df['active_months'] = df['spot'].apply(lambda x: len(FutureContract.from_spot(x).active_months))
df['active_hours'] = df['spot'].apply(lambda x: (x.trading_end_time - x.trading_start_time).seconds/60/60)
df['description'] = df['spot'].apply(lambda x: x.description)
df = df.sort_values(by=['avg_tr_fr_tr']).astype({'best': 'float'}).reset_index(drop=True)
df = df[['spot', 'description', 'avg_tr_fr_tr', 'avg_te_fr_tr', 'test_train_ratio',
         'avg_max_test_regret', 'best', 'pct_best', 'pct_curr_best_is_next_best']]

df['description'] = df['description'].apply(lambda x: x.replace('&', r'\&').replace('#', '\#'))
df['description'] = df['description'].apply(lambda x: format_lines(x, 20))

df = df.rename(columns={'avg_tr_fr_tr': r'Train $R^2$', 'avg_te_fr_tr': r'Test $R^2$', 'avg_max_test_regret': 'Test Regret',
                        'pct_curr_best_is_next_best': 'Stability',
                        'pct_best': r'$\hat{\delta}$ Freq', 'best': r'$\hat{\delta}$',
                        'spot': 'Spot', 'description': 'Description',
                        'test_train_ratio': 'TT Ratio'})


file_path = f'./tables/appendix/all_r2_{train_months}_{test_months}.tex'
caption = f"Average $R^2$ results for best model with {train_months}m train, {test_months}m test."
label = f'table:apx_r2_{train_months}_{test_months}'

with pd.option_context("max_colwidth", 1000):
    df.to_latex(file_path, index=False, float_format="%.3f", longtable=True, caption=caption, label=label, escape=False)

In [21]:
df

,Spot,Description,Train $R^2$,Test $R^2$,TT Ratio,Test Regret,$\hat{\delta}$,$\hat{\delta}$ Freq,Stability
0,ES,\makecell{S\&P 500 E-Mini\\Futures},0.402570,0.388908,0.972031,0.008141,0.10,0.418605,0.523810
1,HI,\makecell{Hang Seng Index\\Futures},0.441864,0.435511,0.990617,0.002720,0.35,0.325581,0.595238
2,XX,\makecell{EURO STOXX 50 Index\\Futures},0.459254,0.438253,0.965245,0.017469,0.10,0.883721,0.833333
3,VX,VIX Futures,0.495913,0.484193,0.977955,0.004288,0.10,0.682927,0.725000
4,MI,\makecell{S\&P 400 MidCap\\E-Mini Futures},0.507346,0.495263,0.977674,0.002562,0.45,0.302326,0.452381
...,...,...,...,...,...,...,...,...,...
56,CN,Corn Futures,0.755070,0.754856,1.002523,0.000192,0.10,0.692308,0.750000
57,GO,\makecell{Low Sulphur Gasoil\\Futures ICE},0.768233,0.761026,0.992800,0.002940,0.15,0.279070,0.380952
58,AX,\makecell{Australian 10-Year\\Bond Futures},0.770674,0.765205,0.993582,0.000000,0.10,1.000000,1.000000
59,TU,\makecell{US 2-Year T-Note\\Futures},0.775369,0.759069,0.980480,0.000159,0.10,1.000000,1.000000


: 